In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os

from utils.audio import Audio
from utils.hparams import HParam

from model.embedder import SpeechEmbedder
from datasets.dataloader import create_dataloader

hp = HParam("config.yaml")
audio = Audio(hp)

/root/voicefilter/utils/hparams.py:18: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


# Trainer setup

In [1]:
import os
import time
import logging
import argparse

from utils.trainer import trainer
from utils.hparams import HParam
from utils.writer import MyWriter
from datasets.dataloader import create_dataloader

/root/.virtualenvs/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('-b', '--base_dir', type=str, default='.',
                    help="Root directory of run.")
parser.add_argument('-c', '--config', type=str, required=True,
                    help="folder contain yaml files for configuration")
parser.add_argument('-e', '--embedder_path', type=str, required=True,
                    help="path of embedder model pt file")
parser.add_argument('--checkpoint_path', type=str, default=None,
                    help="path of checkpoint pt file")
parser.add_argument('-m', '--model', type=str, required=True,
                    help="Name of the model. Used for both logging and saving checkpoints.")
parser.add_argument('--train_set', type=str, nargs="+",
                    help="List of dataset subset name for train set.")
parser.add_argument('--test_set', type=str, nargs="+",
                    help="List of dataset subset name for train set.")
args = parser.parse_args([
    "-c", "config.yaml"
    ,"-e", "embedder.pt"
    ,"-m", "performance_prof"
    ,"--train_set", "librispeech-train"
    ,"--test_set", "librispeech-test"
])

hp = HParam(args.config)
with open(args.config, 'r') as f:
    # store hparams as string
    hp_str = ''.join(f.readlines())

pt_dir = os.path.join(args.base_dir, hp.log.chkpt_dir, args.model)
os.makedirs(pt_dir, exist_ok=True)

log_dir = os.path.join(args.base_dir, hp.log.log_dir, args.model)
os.makedirs(log_dir, exist_ok=True)

chkpt_path = args.checkpoint_path if args.checkpoint_path is not None else None

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir,
            '%s-%d.log' % (args.model, time.time()))),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

writer = MyWriter(hp, log_dir)

trainloader = create_dataloader(hp, "generate", dataset_detail=args.train_set, scheme="train")
testloader = create_dataloader(hp, "generate", dataset_detail=args.test_set, scheme="test")

/root/voicefilter/utils/hparams.py:18: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


# Profiling

In [3]:
from torch.profiler import profile, record_function, ProfilerActivity

In [4]:
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True, use_cuda=True) as prof:
    trainer(args, pt_dir, chkpt_path, trainloader, testloader, writer, logger, hp, hp_str, profiling=True)

/root/.virtualenvs/venv/lib/python3.8/site-packages/torch/profiler/profiler.py:228: UserWarning: use_cuda is deprecated, use activities argument instead
  warn("use_cuda is deprecated, use activities argument instead")
2022-04-02 18:33:19,021 - INFO - Starting new training run
2022-04-02 18:33:21,829 - INFO - Wrote summary at step 1
2022-04-02 18:33:22,585 - INFO - Wrote summary at step 2
2022-04-02 18:33:23,334 - INFO - Wrote summary at step 3


In [5]:
prof.export_chrome_trace("trace.json")